In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential, load_model



In [3]:
#loading the model and the scaler and one hot encoder file
model= load_model('model.h5')

with open('one_hot_encoder_geography.pkl','rb') as file:
     one_hot_encoder_geo1=pickle.load(file)

with open('label_encode_gender.pkl','rb') as file:
     lab_encode_gender=pickle.load(file)



In [4]:
with open('scalling.pkl','rb') as file:
     scalling_new=pickle.load(file)  
             

In [5]:
#input data
input_data={
    'CreditScore': 600,
    'Geography':'France',
    'Gender':'male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember': 1,
    'EstimatedSalary': 500000

}

In [6]:
geography_encoded= one_hot_encoder_geo1.transform([[input_data['Geography']]]).toarray()
geo_encoded_df= pd.DataFrame(geography_encoded,columns=one_hot_encoder_geo1.get_feature_names_out(['Geography']))

d:\churn_model_rnn\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [7]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df= pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,male,40,3,60000,2,1,1,500000


In [9]:
#combining data
input_data_df=pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,male,40,3,60000,2,1,1,500000,1.0,0.0,0.0


In [10]:
input_data_df['Gender']



0    male
Name: Gender, dtype: object

In [11]:
input_data_df['Gender']=lab_encode_gender.transform(input_data_df['Gender'])

ValueError: y contains previously unseen labels: 'male'

In [12]:
input_data_df['Gender']=1

In [13]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,500000,1.0,0.0,0.0


In [14]:
input_data_df=input_data_df.drop("Geography",axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,500000,1.0,0.0,0.0


In [15]:
#scallig
input_after_scalling=scalling_new.transform(input_data_df)
input_after_scalling

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699,  6.94716943,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
prediction=model.predict(input_after_scalling)
prediction

1/1 [==============================] - 0s 104ms/step


array([[0.00796813]], dtype=float32)

In [17]:
prediction_probab=prediction[0][0]
prediction_probab

0.00796813

In [18]:
if prediction_probab>0.5:
    print("Customer likely to churn")
else:
     print("Customer not likely to churn")    

Customer not likely to churn
